# EDA Project

In this notebook, you should perform EDA (Exploratory data analysis) on given dataset: real_estate_dataset.csv

We do not want to give you precise steps to follow, but based on previous lessons, you should have an idea what steps are needed and which should not be skipped. In order to guide you just a bit, here are some ideas:

* it is always good to check how many data are missing - and how can we solve missing data in this dataset?
* what data types have different columns?
* what could be set as index? And does it make sense here?
* price column can be for sure better formated
* can we spot some outliers? What method to see outliers can be used?
* Total area can be in metres (we are in Austria) and numeric, loft size as well
* who is the most active broker?
* what is most sold object type?
* can you do nice visualisation of the data?
* is there a correlation between price and size? Can we see it?
* is there any other patterns to discribe?

## What we want to be submitted:
* send us your notebook on github as link and we will give you feedback
* comment everything - explain your thoughts, why you think this column should be dropped, why you did this visualisation, everything.
* you do not have to follow all ideas above, but your analysis should be going from start to end with logical steps
* try to summarize with at least 3 sentences a conclusion on what we can tell about dataset.

https://towardsdatascience.com/how-to-change-semi-structured-text-into-a-pandas-dataframe-ef531d6baab4
https://medium.com/analytics-vidhya/apartment-market-web-scrapping-and-eda-using-python-a9eacf6d64e2
https://github.com/hw355/eda_realtor/blob/main/EDA_Realtor.ipynb
https://medium.com/web-mining-is688-spring-2021/assignment-1-71368059c638
https://stackoverflow.com/questions/56770982/how-to-split-text-in-one-column-of-pandas-data-frame-to-required-formatthree-co

In [ ]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno
%matplotlib inline  
#the output of plotting commands is displayed inline, directly below the code cell that produced it.

In [ ]:
#Load dataset
realstate = pd.read_csv("real_estate_dataset.csv")

In [ ]:
#Peek into the data. I can see the data is troublesome, as there is a lot of information per column
print("Contents of the 'head' of Dataframe : ")
realstate.head

In [ ]:
realstate.columns

In [ ]:
realstate.shape

## DataTypes Right now it is not useful, I need to change the types ¿manually? 

In [ ]:
dataTypeDf = pd.DataFrame(realstate.dtypes.value_counts()).reset_index().rename(columns={"index":"variableType",0:"count"})
fig,ax = plt.subplots()
fig.set_size_inches(20,5)
sns.barplot(data=dataTypeDf,x="variableType",y="count",ax=ax,color="#34495e")
ax.set(xlabel='Variable Type', ylabel='Count',title="Variables Count Across Datatype")

## Missing Values... 
Surely, I should tidy up the dataset to convey any message about it. From a "first glance" point of view I can see there are many missing values and data inside each column is not uniformly reported, but there are few attributes to organize...

What is the percentage of null values per column? In order to do this, I must fill with NaN everywhere there's a missing value. 

In [ ]:
#realstatefilled = realstate.fillna(value=np.nan) #https://stackoverflow.com/questions/23743460/replace-none-with-nan-in-pandas-dataframe
realstatefilled = realstate.replace(r'^\s+$', np.nan, regex=True) #https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas

In [ ]:
realstatefilled.head

I found about this library that sum up for the NaN values in a visual way, I'll make use of the basic graphs. 

In [ ]:
np.round((realstatefilled.isna().sum() / realstatefilled.shape[0]) * 100, 2)

In [ ]:
missingValueColumns = realstatefilled.columns[realstatefilled.isnull().any()].tolist()
#msno.bar(realstatefilled[missingValueColumns],
#         figsize=(16,6),
#         color="#028A0F",
#         fontsize=8,
#         labels=True)
msno.bar(realstatefilled,
         figsize=(16,6),
         color="#028A0F",
         fontsize=8,
         labels=True)

In [ ]:
msno.matrix(realstatefilled[missingValueColumns],
            width_ratios=(10,1),\
            figsize=(16,6),
            color=(0,0, 0),
            fontsize=8,
            sparkline=True,
            labels=True)

In [ ]:
 msno.heatmap(realstatefilled,figsize=(16,16))

In [ ]:
np.round((realstatefilled.isna().sum() / realstatefilled.shape[0]) * 100, 2)
#np.round((realstate.isna().sum() / realstate.shape[0]) * 100, 2)

Columns: Owner Name, mailing Address, email, secondary broker, phone, 2 and email (from the second broker, I presume) have a significant amount of null values. I presume that the absence of information doesn't not imply an information loss, as these may be casual. The important information here is the propierties of the pontential property to buy, the broker and the price... 

The other fields that contain a small percentage of missing values, might not be specially problematic. I have not decide if I will discard the items that don't have info.

In [ ]:
# Delete columns containing either 70% or more than that NaN Values
perc = 70.0
min_count =  int(((100-perc)/100)*realstatefilled.shape[0] + 1)
mod_df = realstatefilled.dropna( axis=1, 
                thresh=min_count)
print("Modified Dataframe : ")
print(mod_df)

In [ ]:
np.round((mod_df.isna().sum() / mod_df.shape[0]) * 100, 2)

### I still have an important amount of columns that have NA values. For instance, Total area, number of stories and year built. If the dataset will serve to predict 

In [ ]:
#realstate.describe()
mod_df.describe()

In [ ]:
#print(realstate.iloc[32:42, 1:11 ])
print(mod_df.iloc[32:42, 1:11 ])

In [ ]:
#realstate[realstate.columns[6]] = realstate[realstate.columns[6]].replace('[\$,]', '', regex=True)
realstate['Price'] = pd.to_numeric(realstate['Price'].replace('[^0-9\.-]','', regex = True))
#realstate['Price'].describe()
#realstate['Price'].describe().apply(lambda x: format(x, 'f'))

In [ ]:
#https://www.w3resource.com/python-exercises/python-basic-exercise-61.php
d_ft = int(input("Input distance in feet: "))
d_inches = d_ft * 12
d_yards = d_ft / 3.0
d_miles = d_ft / 5280.0


df.loc[df.Weight == "155", "Name"] = "John"

###https://www.geeksforgeeks.org/python-pandas-series-str-find/
##https://www.shanelynn.ie/pandas-iloc-loc-select-rows-and-columns-dataframe/
  
# dropping null value columns to avoid errors
data.dropna(inplace = True)
 
# substring to be searched
sub ='a'
 
# creating and passing series to new column
data["Indexes"]= data["Name"].str.find(sub)
 
# display
data

In [ ]:
realstate['size(sqft)'] = realstate['size(sqft)'].astype(int)
realstate['size(sqft)'].describe().apply(lambda x: format(x, 'f'))
realstate['longitude'] = realstate['longitude'].astype(float)
realstate['latitude'] = realstate['latitude'].astype(float)
realstate['last_update'] = pd.to_datetime(realstate['last_update'], format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')

In [ ]:
realstate.info()
#realstate.describe().apply(lambda s: s.apply('{0:.2f}'.format))

In [ ]:
#https://stackoverflow.com/questions/56770982/how-to-split-text-in-one-column-of-pandas-data-frame-to-required-formatthree-co

https://stackoverflow.com/questions/56770982/how-to-split-text-in-one-column-of-pandas-data-frame-to-required-formatthree-co### Images and URL lead to a web page. 
I've been researching about it and I could apply an easy and fast way to see if the webpage is available,
with the package webbrowswer


In [ ]:
## import webbrowser
## url_1 = realstate.url[0]
## webbrowser.open_new_tab(url_1)
##
##

In [ ]:
import pandas as pd

ages = ['Age 96 - 97', 'Age greater than 99', '65-74 years old', '75+ years old']
df = pd.DataFrame({'age': ages})

df[['age_low', 'age_high']] = df['age'].str.extract(r'(\d+)(?:\D+(\d+))?')
print(df)

#### Try r'(\d+)(?:[\s-]*(\d+))?' or r'(\d+)[\s-]*(\d*)' – 


In [ ]:
from datetime import datetime
# Pongo bien la fecha
data['x'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in data['x']]
data['x'] = pd.to_datetime(data['x'])

data.head()


### Type, broker, place seem like correct attributes to organize the data around. Statistics about those should be interesting, the intersection between them and Price, Are, Year built are key info